In [1]:
from sqlalchemy import extract, desc
import datetime
import time
import MetaTrader5 as mt5
import pandas as pd

import options_estrategies as opt_est
from options_estrategies import Option_Due, InformationType
import models as model

In [2]:
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()

estrategies = opt_est.options_estrategies(model, model.PETR4, model.PETR4_OPTIONS)

In [25]:
def thl_convert_dict_from_update_ticks_to_dataframe(tick_data):
    df_tick_data = pd.DataFrame.from_dict(tick_data)
    df_tick_data = df_tick_data.drop(columns=['_sa_instance_state','timestamp_option','days_to_due_date'])
    df_tick_data['stock_price'] = df_tick_data.stock_price.astype(float)
    df_tick_data = df_tick_data[['option_name','deal_type_zone','stock_price','strike','bid','ask']]
    
    return df_tick_data


In [31]:
updated_ticks = estrategies.update_quotes(just_last_update=True,mode=InformationType.Offline)
options_dataframe_list = []
for ticks_to_process in updated_ticks:
    options_updated = []
    for i in ticks_to_process:
        options_updated.append(i.__dict__)

    df_options_updated = thl_convert_dict_from_update_ticks_to_dataframe(options_updated)
    df_options_updated.reset_index()
    df_options_updated = df_options_updated.set_index('strike')
    options_dataframe_list.append(df_options_updated)

In [32]:
df_this_month = options_dataframe_list[0]
df_next_month = options_dataframe_list[1]
df_after_next_month = options_dataframe_list[2]

In [95]:
df_thl = pd.merge(df_this_month, df_next_month, left_index=True, right_index=True)
df_thl_next_month = pd.merge(df_next_month, df_after_next_month, left_index=True, right_index=True)
df_thl_calendar = pd.merge(df_this_month, df_after_next_month, left_index=True, right_index=True)

In [96]:
def thl_convert_dict_from_update_ticks_to_dataframe(data_thl):
    df_thl_compare = data_thl.copy()
    df_thl_compare["thl_cost"] = (df_thl_compare['bid_x'] - df_thl_compare['ask_y'])*-1
    df_thl_compare["thl_percent_of_strike"] = (df_thl_compare["thl_cost"] / df_thl_compare.index)*100
    df_thl_compare = df_thl_compare[['option_name_x','option_name_y','thl_percent_of_strike','thl_cost','bid_x','ask_y','deal_type_zone_y']]
    df_thl_compare = df_thl_compare[df_thl_compare['ask_y'] > 0]
    return df_thl_compare

In [97]:
comparison_this_month = compare_to_dataframes_of_thl(df_thl)
comparison_next_month = compare_to_dataframes_of_thl(df_thl_next_month)
comparison_calendar = compare_to_dataframes_of_thl(df_thl_calendar)

In [98]:
good_operation_THL = comparison_this_month[comparison_this_month['thl_percent_of_strike'] < 1.5]
good_operation_THL_next_month = comparison_next_month[comparison_next_month['thl_percent_of_strike'] < 1.5]
good_operation_calendar = comparison_calendar[comparison_calendar['thl_percent_of_strike'] < 1.5]

In [99]:
if len(good_operation_THL.index) > 0:
    print(good_operation_THL)
    
if len(good_operation_THL_next_month.index) > 0:
    print(good_operation_THL_next_month)

if len(good_operation_calendar.index) > 0:
    print(good_operation_calendar)    

       option_name_x option_name_y  thl_percent_of_strike  thl_cost  bid_x  \
strike                                                                       
28.96       PETRH243      PETRI289               1.450276      0.42   0.49   

        ask_y deal_type_zone_y  
strike                          
28.96    0.91              OTM  


In [103]:
return_of_funcion = dict(thl=comparison_this_month, thl_next_month=comparison_next_month, thl_calendar=comparison_calendar)

In [105]:
return_of_funcion['thl']

,option_name_x,option_name_y,thl_percent_of_strike,thl_cost,bid_x,ask_y,deal_type_zone_y
strike,,,,,,,
26.46,PETRH254,PETRI271,2.683296,0.71,1.53,2.24,ITM
27.21,PETRH261,PETRI279,2.609335,0.71,1.19,1.90,OTM
27.46,PETRH256,PETRI281,2.804079,0.77,1.03,1.80,OTM
27.71,PETRH284,PETRI278,1.840491,0.51,0.94,1.45,OTM
27.96,PETRH247,PETRI286,1.967096,0.55,0.84,1.39,OTM
28.21,PETRH289,PETRI254,1.878766,0.53,0.71,1.24,OTM
28.46,PETRH279,PETRI276,2.037948,0.58,0.67,1.25,OTM
28.96,PETRH243,PETRI289,1.450276,0.42,0.49,0.91,OTM
29.21,PETRH299,PETRI291,2.259500,0.66,0.43,1.09,OTM
